In [1]:
import numpy as np 
import pandas as pd

from sklearn.model_selection import StratifiedKFold , cross_validate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline 

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv("emails.csv")

x = df["text"]
y = df["spam"]

In [3]:
log_reg = LogisticRegression(max_iter = 1000 , solver = "liblinear" , random_state = 42)

ran_for = RandomForestClassifier(n_estimators = 200 , max_depth = None , random_state = 42 , n_jobs = -1)


In [6]:
LR_pipeline = Pipeline([("tfidf" , TfidfVectorizer(stop_words = "english " , max_df = 0.9)), ("clf" , log_reg)])

In [7]:
rf_pipeline = Pipeline([("tfidf" , TfidfVectorizer(stop_words ="english" , max_df = 0.9)), ("clf" ,ran_for)])

In [22]:
scoring = {
    "accuracy": "accuracy",
    "precision": "precision",
    "recall": "recall",
    "f1": "f1",
    "roc_auc": "roc_auc"
}


In [23]:
def evaluate_model(model, X, y, n_splits):
    cv = StratifiedKFold(
        n_splits=n_splits,
        shuffle=True,
        random_state=42
    )

    results = cross_validate(
        model,
        x,
        y,
        cv=cv,
        scoring=scoring,
        n_jobs=-1
    )

    summary = {
        metric: (
            np.mean(results[f"test_{metric}"]),
            np.std(results[f"test_{metric}"])
        )
        for metric in scoring
    }

    return summary


In [26]:
models = {
    "Logistic Regression": LR_pipeline,
    "Random Forest": rf_pipeline
}

for name, model in models.items():
    print(f"\n{name}")

    for k in [5, 10]:
        scores = evaluate_model(model, x, y, n_splits=k)

        print(f"\n{k}-Fold CV Results:")
        for metric, (mean, std) in scores.items():
            print(f"{metric:10s}: {mean:.4f} ± {std:.4f}")



Logistic Regression


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\BD\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\BD\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "C:\Users\BD\AppData\Local\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 654, in fit
    Xt = self._fit(X, y, routed_params, raw_params=params)
  File "C:\Users\BD\AppData\Local\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 588, in _fit
    

In [ ]:
print(LR_pipeline , rf_pipeline)